# THỐNG KÊ DỮ LIỆU

1. Yêu cầu chung: Thống kê được số lượng ảnh trong từng phân loại mà mỗi SV đóng góp

2. Yêu cầu cụ thể:
- Input:
    + Thư mục cha chứa các thư mục con - mỗi thư mục con tương ứng với tên của từng hiệu xe (Honda, Suzuki, VinFast, Others). Ví dụ: https://drive.google.com/drive/u/1/folders/1Uj0V9URNHpzSHeXHSB89AoGCjGki8Yra
    + Các ảnh được đặt tên theo quy ước: các tập tin ảnh theo quy ước https://colab.research.google.com/drive/1bUmXMM_ggnEXKo2qylfe6h0JnQuos-8_
- Output:
    + File CarDataset-1.csv - Thống kê số lượng ảnh theo từng SV
      + Mỗi dòng sẽ có các thông tin cách nhau bằng dấu phẩy, theo quy ước: MSSV, All, Số lượng.
        
        Ví dụ: 20221234, All, 1.2
      + Số lượng có thể là số thực vì một ảnh có thể có đóng góp từ nhiều sinh viên
    + File CarDataset-2.csv - Thống kê số lượng ảnh theo từng Hiệu xe mà từng SV đóng góp
      + Mỗi dòng sẽ có các thông tin cách nhau bằng dấu phẩy, theo quy ước: MSSV, Hiệu xe, Số lượng.
        
        Ví dụ: 20221234, Honda, 1.2
      + Số lượng có thể là số thực vì một ảnh có thể có đóng góp từ nhiều sinh viên
      + Chỉ chấp nhận file .jpg, .jpeg và .png
3. Nộp bài: SV share notebook. Các bài nộp sớm sẽ được full điểm. Deadline: 17:00 - ??/??/2024

4. Bài làm đạt yêu cầu sẽ được paste vào notebook với ghi nhận đóng góp từ tác giả.


## **Thông tin của tác giả, ngày cập nhật**

<hr>

**Thành viên nhóm**:
- **Trần Đình Khánh Đăng - 22520195**
- **Tăng Nhất - 22521027**
- **Lê Minh Nhựt - 22521060**

**Ngày cập nhật**: 13/12/2024

## Import thư viện cần thiết

In [1]:
import re
import os
import csv
from tqdm import tqdm
import pandas as pd
from collections import defaultdict

### Mount Drive Colab

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Code

In [3]:
def create_dataframe(data, sort=False, sort_value=None) -> pd.DataFrame:
    """
    Tạo một DataFrame từ danh sách dữ liệu, với tùy chọn sắp xếp theo cột cụ thể.

    Args:
        data (list of dict): Danh sách dữ liệu để chuyển thành DataFrame.
        sort (bool, optional): Nếu True, sắp xếp DataFrame theo giá trị trong cột `sort_value`.
        sort_value (str, optional): Tên cột dùng để sắp xếp (chỉ áp dụng khi `sort` là True).

    Returns:
        pd.DataFrame: DataFrame đã được tạo và sắp xếp (nếu cần).
    """
    df = pd.DataFrame(data)
    if sort and sort_value:
        df = df.sort_values(by=sort_value, ascending=False).reset_index(drop=True)
    return df

def get_image_count(base_dir='./',
                   categories=['Honda', 'Hyundai', 'KIA', 'Mazda', 'Mitsubishi', 'Others', 'Suzuki', 'Toyota', 'VinFast'],
                   take_average=False,
                   ) -> defaultdict:
    """
    Lấy số lượng ảnh theo MSSV và loại xe từ thư mục gốc.

    Args:
        base_dir (str, optional): Thư mục gốc chứa các thư mục loại xe.
        categories (list of str, optional): Danh sách các loại xe hợp lệ.
        take_average (bool, optional): Nếu True, chia đều số lượng ảnh cho các thành viên trong nhóm MSSV.

    Returns:
        defaultdict: Dictionary dạng {MSSV: {Hiệu xe: Số lượng ảnh}}.
    """
    # Tạo biến tạm để lưu số lượng ảnh theo MSSV
    image_count = defaultdict(lambda: defaultdict(float))  # Có tình trạng có MSSV không đóng góp ảnh (?)

    student_ids_pattern = r'(\d{8}(?:-\d{8})*)' # Lấy MSSV hợp lệ (đủ 8 số)
    categories_pattern = '|'.join(categories) # Lấy hiệu xe hợp lệ
    file_extension_pattern = r'\.\d+\.(jpg|jpeg|png)$' # Lấy extension hợp lệ (chỉ chấp nhận file .jpg, .jpeg và .png)
    # Regex lấy tên file hợp lệ
    accepted_filename = re.compile(fr'{student_ids_pattern}\.({categories_pattern}){file_extension_pattern}')

    # accepted_filename_2 = re.compile(fr'(\d{{8}}(?:-\d{{8}})*)\.({categories_pattern})\.\d+\.(jpg|jpeg|png)$')
    for category in categories: # Duyệt qua các hiệu xe
        category_path = os.path.join(base_dir, category)
        if os.path.isdir(category_path): # Kiểm tra nếu thư mục tồn tại
            for filename in os.listdir(category_path):
                match = accepted_filename.match(filename)
                if match: # Chỉ xử lý file có tên hợp lệ
                    student_ids, car_category, img_idx = match.groups()
                    student_ids_list = student_ids.split('-')
                    num_members = len(student_ids_list)
                    # Đếm ảnh theo MSSV
                    for student_id in student_ids_list:
                        image_count[student_id][car_category] += 1 / num_members if take_average else 1

    return image_count

def write_csv(base_dir='./',
              dataset_dir='./',
              categories=['Honda', 'Hyundai', 'KIA', 'Mazda', 'Mitsubishi', 'Others', 'Suzuki', 'Toyota', 'VinFast'],
              file_name_cars='CarDataset-1.csv',
              file_name_categories='CarDataset-2.csv',
              save_csv=False,
              take_average=False,
              sort=False
              ) -> tuple:
    """
    Ghi số liệu ảnh theo MSSV và loại xe vào các file CSV.

    Args:
        base_dir (str, optional): Thư mục gốc chứa các thư mục loại xe.
        dataset_dir (str, optional): Thư mục lưu trữ các file CSV đầu ra.
        categories (list of str, optional): Danh sách các loại xe hợp lệ.
        file_name_cars (str, optional): Tên file CSV lưu số lượng ảnh tổng hợp theo MSSV.
        file_name_categories (str, optional): Tên file CSV lưu số lượng ảnh chi tiết theo loại xe.
        save_csv (bool, optional): Nếu True, lưu các DataFrame dưới dạng file CSV.
        take_average (bool, optional): Nếu True, chia đều số lượng ảnh cho các thành viên trong nhóm MSSV.
        sort (bool, optional): Nếu True, sắp xếp dữ liệu theo số lượng ảnh.

    Returns:
        tuple: Tuple chứa 2 DataFrame: (df_cars, df_categories).
    """
    os.makedirs(dataset_dir, exist_ok=True) # Tạo thư mục đầu ra nếu chưa tồn tại
    image_count = get_image_count(base_dir=base_dir, categories=categories, take_average=take_average) # Lấy số lượng ảnh theo MSSV và loại xe

    cars_list = [] # Danh sách tổng số lượng ảnh của mỗi MSSV
    categories_list = [] # Danh sách số lượng ảnh theo hiệu xe của mỗi MSSV

    for student_id, car_data in image_count.items():
        total_images = 0
        for car_category, count in car_data.items():
            total_images += count
            categories_list.append({'MSSV': student_id, 'Hiệu xe': car_category, 'Số lượng': round(count, 2)})
        cars_list.append({'MSSV': student_id, 'All': 'All', 'Số lượng': round(total_images, 2)})

    # Chuyển danh sách thành DataFrame
    df_cars = create_dataframe(cars_list, sort=sort, sort_value='Số lượng')
    df_categories = create_dataframe(categories_list, sort=sort, sort_value='Số lượng')

    # Lấy đường dẫn file CSV
    file_path_categories = os.path.join(dataset_dir, file_name_categories)
    file_path_cars = os.path.join(dataset_dir, file_name_cars)

    # Lưu CSV nếu cần
    if save_csv:
        df_categories.to_csv(file_path_categories, index=False)
        df_cars.to_csv(file_path_cars, index=False)

        print(f'{file_name_cars} saved to: {file_path_cars}')
        print(f'{file_name_categories} saved to: {file_path_categories}')

    return df_cars, df_categories


In [4]:
base_dir = './drive/MyDrive/Classroom/CS114.ML - MACHINE LEARNING CS114.P11 - HK1 (2024-2025)/Final Project/Public'
dataset_dir = './drive/MyDrive/Classroom/CS114.ML - MACHINE LEARNING CS114.P11 - HK1 (2024-2025)/Final Project/Dataset'

categories = ['Others', 'Honda', 'Hyundai', 'KIA', 'Mazda', 'Mitsubishi', 'Suzuki', 'Toyota', 'VinFast']

file_name_cars = 'CarDataset-1.csv'
file_name_categories = 'CarDataset-2.csv'

df_cars, df_categories = write_csv(base_dir=base_dir,
                                   dataset_dir=dataset_dir,
                                   categories=categories,
                                   file_name_cars=file_name_cars,
                                   file_name_categories=file_name_categories,
                                   save_csv=False,
                                   take_average=False,
                                   sort=True)   

CarDataset-1.csv saved to: ./drive/MyDrive/Classroom/CS114.ML - MACHINE LEARNING CS114.P11 - HK1 (2024-2025)/Final Project/Dataset/CarDataset-1.csv
CarDataset-2.csv saved to: ./drive/MyDrive/Classroom/CS114.ML - MACHINE LEARNING CS114.P11 - HK1 (2024-2025)/Final Project/Dataset/CarDataset-2.csv


In [5]:
print(f'{file_name_cars}:\n{df_cars}')

CarDataset-1.csv:
        MSSV  All  Số lượng
0   22521060  All   11969.0
1   22521027  All   11969.0
2   22520195  All   11969.0
3   22520459  All    3777.0
4   22520507  All    3777.0
5   22520862  All    3777.0
6   22520394  All    2945.0
7   22520395  All    2945.0
8   22520779  All    2720.0
9   22521587  All    2720.0
10  22521070  All    2183.0
11  22520211  All    2183.0
12  22520970  All    1764.0
13  22521342  All    1764.0
14  22521586  All    1446.0
15  22521559  All    1446.0
16  21520938  All    1046.0
17  22520634  All    1046.0
18  22521692  All    1035.0
19  22521676  All    1035.0
20  22520915  All     660.0
21  22520914  All     660.0
22  22521599  All     599.0
23  22521492  All     599.0
24  21522924  All     596.0
25  22520477  All     578.0
26  22520490  All     578.0
27  22521614  All     499.0
28  22521560  All     499.0
29  22520348  All     419.0
30  22520530  All     419.0
31  22520837  All     419.0
32  22521463  All     299.0
33  22521213  All     299.0
34

In [6]:
print(f'{file_name_categories}:\n{df_categories}')

CarDataset-2.csv:
         MSSV  Hiệu xe  Số lượng
0    22520507   Suzuki    2400.0
1    22520862   Suzuki    2400.0
2    22520459   Suzuki    2400.0
3    22520195   Suzuki    2307.0
4    22521027   Suzuki    2307.0
..        ...      ...       ...
346  22521425      KIA      10.0
347  22520318    Mazda      10.0
348  22520318    Honda      10.0
349  22520318      KIA      10.0
350  22520318  Hyundai      10.0

[351 rows x 3 columns]
